# Thu thập dữ liệu Tỷ giá & Lãi suất

In [68]:
import datetime

import pandas as pd
import pandas_datareader as web

import yfinance as yf


## Tỷ giá USD/VND, USD/CNY

Thu thập bằng [Yahoo finance](https://finance.yahoo.com/) - `yfinance`

In [ ]:
# Kiểm tra ngày bắt đầu có dữ liệu

ticker = 'USDVND=X'
# ticker = 'USDCNY=X'

df = yf.download(ticker, start='1900-01-01')

first_date = df.index[0]
print(f'Ngày đầu tiên {ticker}: {first_date}')


[*********************100%***********************]  1 of 1 completed

Ngày đầu tiên USDVND=X: 2003-12-01 00:00:00


Lấy tất cả dữ liệu từ trước đến nay

In [ ]:
# Cách 1:
start_date = first_date
end_date = datetime.date.today()

df = yf.download(ticker, start=start_date, end=end_date)
df.head()


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,USDVND=X,USDVND=X,USDVND=X,USDVND=X,USDVND=X
Date,,,,,
2003-12-01,15113.0,15113.0,15113.0,15113.0,0
2003-12-02,15075.0,15075.0,15075.0,15075.0,0
2003-12-03,15102.0,15102.0,15102.0,15102.0,0
2003-12-04,15087.0,15087.0,15087.0,15087.0,0
2003-12-05,15089.0,15089.0,15089.0,15089.0,0


In [ ]:
# Cách 2:
# df = yf.download(ticker, period="max")
# df.head()


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5548 entries, 2003-12-01 to 2025-03-21
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   (Close, USDVND=X)   5548 non-null   float64
 1   (High, USDVND=X)    5548 non-null   float64
 2   (Low, USDVND=X)     5548 non-null   float64
 3   (Open, USDVND=X)    5548 non-null   float64
 4   (Volume, USDVND=X)  5548 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 260.1 KB


Thư viện `yfinance` phiên bản mới đã thay đổi cấu trúc dataframe. Cần tiền xử lý trước khi sử dụng.


In [ ]:
# Cách 1: Xóa level columns không mong muốn
df.columns = df.columns.droplevel(1)

# Cách 2: Lấy level 0 của MultiIndex làm index cho columns
# df.columns = [col[0] for col in df.columns]

df.head()


Price,Close,High,Low,Open,Volume
Date,,,,,
2003-12-01,15113.0,15113.0,15113.0,15113.0,0
2003-12-02,15075.0,15075.0,15075.0,15075.0,0
2003-12-03,15102.0,15102.0,15102.0,15102.0,0
2003-12-04,15087.0,15087.0,15087.0,15087.0,0
2003-12-05,15089.0,15089.0,15089.0,15089.0,0


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5548 entries, 2003-12-01 to 2025-03-21
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   5548 non-null   float64
 1   High    5548 non-null   float64
 2   Low     5548 non-null   float64
 3   Open    5548 non-null   float64
 4   Volume  5548 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 260.1 KB


In [ ]:
# Chuyển cột Date trở thành dữ liệu bình thường
df = df.reset_index()
df['Date'] = pd.to_datetime(df['Date'])
df.head()

# Đặt lại thành index (nếu cần)
# df["Date"] = pd.to_datetime(df["Date"])
# df = df.set_index("Date")


Price,Date,Close,High,Low,Open,Volume
0,2003-12-01,15113.0,15113.0,15113.0,15113.0,0
1,2003-12-02,15075.0,15075.0,15075.0,15075.0,0
2,2003-12-03,15102.0,15102.0,15102.0,15102.0,0
3,2003-12-04,15087.0,15087.0,15087.0,15087.0,0
4,2003-12-05,15089.0,15089.0,15089.0,15089.0,0


In [ ]:
df = df.drop('Volume', axis=1)
df.head()


Price,Date,Close,High,Low,Open
0,2003-12-01,15113.0,15113.0,15113.0,15113.0
1,2003-12-02,15075.0,15075.0,15075.0,15075.0
2,2003-12-03,15102.0,15102.0,15102.0,15102.0
3,2003-12-04,15087.0,15087.0,15087.0,15087.0
4,2003-12-05,15089.0,15089.0,15089.0,15089.0


In [ ]:
# df.to_csv('USDVND.csv', index=False)


## Thu thập dữ liệu CNY/VND

Thu thập bằng Google Finance (trang tính)

In [75]:
def convert_url(sheet_url: str) -> str:
    """
    Chuyển đổi URL Google Sheets từ dạng chỉnh sửa thành dạng CSV có thể tải xuống.

    :param sheet_url: Đường dẫn Google Sheets gốc
    :return: Đường dẫn CSV để tải dữ liệu
    """
    if "/edit" in sheet_url:
        sheet_id = sheet_url.split("/d/")[1].split("/edit")[0]  # Lấy ID của Google Sheets
        url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"
        return url
    else:
        raise ValueError("URL không hợp lệ. Truy cập vào tệp google sheets, mở quyền truy cập (người xem) và copy đường dẫn.")


In [76]:
url = "https://docs.google.com/spreadsheets/d/1XQdpdxnucpFn4hhoLLh_Np5BFzEP2qX3_KjYWKtrIiU/edit?gid=0"
csv_url = convert_url(url)
print(csv_url)


https://docs.google.com/spreadsheets/d/1XQdpdxnucpFn4hhoLLh_Np5BFzEP2qX3_KjYWKtrIiU/export?format=csv


In [72]:
df = pd.read_csv(csv_url)
df.head()


,Date,Close
0,01/01/2019,"3371,47984"
1,02/01/2019 23:58:00,"3379,59774"
2,03/01/2019 23:58:00,"3374,49506"
3,04/01/2019 23:58:00,"3375,8867"
4,05/01/2019 23:58:00,"3375,8867"


In [ ]:
# df["Date"] = pd.to_datetime(df["Date"], format="mixed").dt.date
df["Date"] = pd.to_datetime(df["Date"], format="mixed", dayfist=True).dt.date
# df["Date"] = pd.to_datetime(df["Date"], format="mixed", dayfirst=True).dt.strftime("%d/%m/%Y")
# df["Date"] = pd.to_datetime(df["Date"], format="mixed", dayfirst=True).dt.strftime("%d/%m/%y")
df.head()


,Date,Close
0,2019-01-01,"3371,47984"
1,2019-02-01,"3379,59774"
2,2019-03-01,"3374,49506"
3,2019-04-01,"3375,8867"
4,2019-05-01,"3375,8867"


Hàm `pd.to_datetime()` mặc định, trích xuất ngày theo định dạng  `mm/dd/yyyy`(Châu Âu). Khi thêm tham số `dayfirst=True` vào `pd.to_datetime()`, pandas sẽ ưu tiên hiểu ngày theo định dạng `dd/mm/yyyy` (Việt Nam).

## Thu thập lãi suất của Fed

Bằng thư viện `pandas_datareader` - API của [FRED (Federal Reserve Economic Data)](https://fred.stlouisfed.org/)<br>

**FRED (Federal Reserve Economic Data)** là cơ sở dữ liệu kinh tế do **Federal Reserve Bank of St. Louis (Ngân hàng Dự trữ Liên bang St. Louis)** cung cấp.

Các [mã lãi suất FED](https://fred.stlouisfed.org/categories/118), một số mã tiêu biểu:
* Effective Federal Funds Rate **([EFFR](https://fred.stlouisfed.org/series/EFFR))**: **Tần suất: hàng ngày**
* Federal Funds Effective Rate **([DFF](https://fred.stlouisfed.org/series/DFF))**: **Tần suất: hàng ngày/ 7 ngày**
* Federal Funds Effective Rate (FEDFUNDS)

*Tham khảo: Các [lãi suất](https://fred.stlouisfed.org/categories/22) khác*

In [74]:
fed_ticker='EFFR'
start = datetime.datetime(2019, 1, 1)
end = datetime.date.today()

df = web.DataReader(fed_ticker, 'fred', start, end)
df.tail()


,EFFR
DATE,
2025-03-14,4.33
2025-03-17,4.33
2025-03-18,4.33
2025-03-19,4.33
2025-03-20,4.33
